<a href="https://colab.research.google.com/github/sreeproject/AI-/blob/main/Brain_Tumor_Sample_ML_Submission_Template_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    -  Brain Tumor MRI Image Classification




##### **Project Type**    - Classification/Supervised
##### **Contribution**    - Individual


# **Project Summary -**

This project aims to develop a deep learning-based solution for classifying brain MRI images into multiple categories according to tumor type. It involves building a custom CNN model from scratch and enhancing performance through transfer learning using pretrained models. The project also includes deploying a user-friendly Streamlit web application to enable real-time tumor type predictions from uploaded MRI images.


# **GitHub Link -**

Provide your GitHub Link here.

# **Problem Statement**


 Brain Tumor MRI Image Classification

# **General Guidelines** : -  

1.   Well-structured, formatted, and commented code is required.
2.   Exception Handling, Production Grade Code & Deployment Ready Code will be a plus. Those students will be awarded some additional credits.
     
     The additional credits will have advantages over other students during Star Student selection.
       
             [ Note: - Deployment Ready Code is defined as, the whole .ipynb notebook should be executable in one go
                       without a single error logged. ]

3.   Each and every logic should have proper comments.
4. You may add as many number of charts you want. Make Sure for each and every chart the following format should be answered.
        

```
# Chart visualization code
```
            

*   Why did you pick the specific chart?
*   What is/are the insight(s) found from the chart?
* Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

5. You have to create at least 15 logical & meaningful charts having important insights.


[ Hints : - Do the Vizualization in  a structured way while following "UBM" Rule.

U - Univariate Analysis,

B - Bivariate Analysis (Numerical - Categorical, Numerical - Numerical, Categorical - Categorical)

M - Multivariate Analysis
 ]





6. You may add more ml algorithms for model creation. Make sure for each and every algorithm, the following format should be answered.


*   Explain the ML Model used and it's performance using Evaluation metric Score Chart.


*   Cross- Validation & Hyperparameter Tuning

*   Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

*   Explain each evaluation metric's indication towards business and the business impact pf the ML model used.




















# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import dataset,dataloader
import glob
import cv2
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Dataset Loading

In [ ]:
root = r'/content/drive/MyDrive/Tumour'
print("dataset exists:", os.path.exists(root))

if os.path.exists(root):
    print("Subfolders:")
    for sub in os.listdir(root):
        print("  ➤", sub)

### Dataset First View

In [ ]:
os.listdir("/content/drive/MyDrive/Tumour/train")

In [ ]:
os.listdir("/content/drive/MyDrive/Tumour/valid")

In [ ]:
os.listdir("/content/drive/MyDrive/Tumour/test")

In [ ]:
import tensorflow as tf
import tensorflow.keras.preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
img=image.load_img('/content/drive/MyDrive/Tumour/train/glioma/Tr-gl_0011_jpg.rf.61e213cb5a0f97fedd1bacd0428c0133.jpg')
plt.imshow(img)

In [ ]:
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import glob

path = r'/content/drive/MyDrive/Tumour/valid/glioma/*.jpg'

files = glob.glob(path)
if files:
    img_path = files[0]  # just pick the first one
    img = image.load_img(img_path)
    plt.imshow(img)
    plt.title(os.path.basename(img_path))
    plt.axis('off')
    plt.show()
else:
    print("No images found in folder.")


In [ ]:
import tensorflow as tf
import tensorflow.keras.preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
img=image.load_img('/content/drive/MyDrive/Tumour/test/glioma/Tr-gl_0016_jpg.rf.99746694ea97fe0b73108832b462d48e.jpg')
plt.imshow(img)

### Dataset Rows & Columns count

In [ ]:
img=image.load_img('/content/drive/MyDrive/Tumour/test/glioma/Tr-gl_0016_jpg.rf.99746694ea97fe0b73108832b462d48e.jpg')
img_array = np.array(img) # Convert PIL Image to NumPy array
print(img_array.shape) # Access the shape of the NumPy array

### Dataset Information

In [ ]:
# Path to images inside extracted folder

data_dir1 = '/content/drive/MyDrive/Tumour/train/*/*.jpg'
train = []
for f in glob.glob(data_dir1, recursive=True):
    img = cv2.imread(f)
    if img is not None:
        train.append(img)
    else:
        print(f"Failed to load: {f}")

print(f"Total images loaded: {len(train)}")

In [ ]:
# Path to images inside extracted folder

data_dir2 = '/content/drive/MyDrive/Tumour/valid/*/*.jpg'
valid = []
for f in glob.glob(data_dir2, recursive=True):
    img = cv2.imread(f)
    if img is not None:
        valid.append(img)
    else:
        print(f"Failed to load: {f}")

print(f"Total images loaded: {len(valid)}")

In [ ]:
# Path to images inside extracted folder

data_dir3 = '/content/drive/MyDrive/Tumour/test/*/*.jpg'
test = []
for f in glob.glob(data_dir3, recursive=True):
    img = cv2.imread(f)
    if img is not None:
        test.append(img)
    else:
        print(f"Failed to load: {f}")

print(f"Total images loaded: {len(test)}")

Train contains 1695 images, valid contains 502 images and test contains 246 images.

In [ ]:
from collections import defaultdict
from PIL import Image
dataset_path = "/content/drive/MyDrive/Tumour/train"  # or any folder containing class subfolders

# Step 1: Collect class-wise image counts and image resolutions
class_counts = defaultdict(int)
image_sizes = []

# Walk through the dataset
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_dir):
        for image_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, image_name)
            try:
                with Image.open(img_path) as img:
                  class_counts[class_name] += 1
                  image_sizes.append(img.size)
            except Exception as e:
                  print(f"Skipping image: {img_path} | Error: {e}")

# Step 2: Convert to DataFrame
df_counts = pd.DataFrame(list(class_counts.items()), columns=['Tumor_Type', 'Image_Count'])
df_sizes = pd.DataFrame(image_sizes, columns=['Width', 'Height'])

# Step 3: Plot Class Distribution
plt.figure(figsize=(8, 5))
sns.barplot(data=df_counts, x='Tumor_Type', y='Image_Count', palette='Set2')
plt.title("Class Distribution (Tumor Types)")
plt.ylabel("Number of Images")
plt.xlabel("Tumor Type")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Step 4: Plot Image Resolution Distribution
plt.figure(figsize=(8, 5))
sns.histplot(df_sizes['Width'], color='skyblue', kde=True, label='Width', bins=30)
sns.histplot(df_sizes['Height'], color='orange', kde=True, label='Height', bins=30)
plt.title("Image Resolution Distribution")
plt.xlabel("Pixels")
plt.legend()
plt.tight_layout()
plt.show()

# Step 5: Print summary statistics
print("Class Distribution:\n", df_counts)
print("\nImage Resolution Stats:\n", df_sizes.describe())

In [ ]:
plt.figure(figsize=(7, 7))
plt.pie(df_counts['Image_Count'], labels=df_counts['Tumor_Type'], autopct='%1.1f%%', startangle=140, colors=sns.color_palette('Set2'))
plt.title("Tumor Class Distribution (Pie Chart)")
plt.tight_layout()
plt.show()

In [ ]:
from collections import defaultdict
from PIL import Image
dataset_path = "/content/drive/MyDrive/Tumour/valid"  # or any folder containing class subfolders

# Step 1: Collect class-wise image counts and image resolutions
class_counts = defaultdict(int)
image_sizes = []

# Walk through the dataset
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_dir):
        for image_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, image_name)
            try:
                with Image.open(img_path) as img:
                  class_counts[class_name] += 1
                  image_sizes.append(img.size)
            except Exception as e:
                  print(f"Skipping image: {img_path} | Error: {e}")

# Step 2: Convert to DataFrame
df_counts1 = pd.DataFrame(list(class_counts.items()), columns=['Tumor_Type', 'Image_Count'])
df_sizes = pd.DataFrame(image_sizes, columns=['Width', 'Height'])

# Step 3: Plot Class Distribution
plt.figure(figsize=(8, 5))
sns.barplot(data=df_counts, x='Tumor_Type', y='Image_Count', palette='Set2')
plt.title("Class Distribution (Tumor Types)")
plt.ylabel("Number of Images")
plt.xlabel("Tumor Type")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Step 4: Plot Image Resolution Distribution
plt.figure(figsize=(8, 5))
sns.histplot(df_sizes['Width'], color='skyblue', kde=True, label='Width', bins=30)
sns.histplot(df_sizes['Height'], color='orange', kde=True, label='Height', bins=30)
plt.title("Image Resolution Distribution")
plt.xlabel("Pixels")
plt.legend()
plt.tight_layout()
plt.show()

# Step 5: Print summary statistics
print("Class Distribution:\n", df_counts1)
print("\nImage Resolution Stats:\n", df_sizes.describe())

In [ ]:
plt.figure(figsize=(7, 7))
plt.pie(df_counts1['Image_Count'], labels=df_counts1['Tumor_Type'], autopct='%1.1f%%', startangle=140, colors=sns.color_palette('Set2'))
plt.title("Tumor Class Distribution (Pie Chart)")
plt.tight_layout()
plt.show()

In [ ]:
from collections import defaultdict
from PIL import Image
dataset_path = "/content/drive/MyDrive/Tumour/test"  # or any folder containing class subfolders

# Step 1: Collect class-wise image counts and image resolutions
class_counts = defaultdict(int)
image_sizes = []

# Walk through the dataset
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_dir):
        for image_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, image_name)
            try:
                with Image.open(img_path) as img:
                  class_counts[class_name] += 1
                  image_sizes.append(img.size)
            except Exception as e:
                  print(f"Skipping image: {img_path} | Error: {e}")

# Step 2: Convert to DataFrame
df_counts2 = pd.DataFrame(list(class_counts.items()), columns=['Tumor_Type', 'Image_Count'])
df_sizes = pd.DataFrame(image_sizes, columns=['Width', 'Height'])

# Step 3: Plot Class Distribution
plt.figure(figsize=(8, 5))
sns.barplot(data=df_counts, x='Tumor_Type', y='Image_Count', palette='Set2')
plt.title("Class Distribution (Tumor Types)")
plt.ylabel("Number of Images")
plt.xlabel("Tumor Type")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Step 4: Plot Image Resolution Distribution
plt.figure(figsize=(8, 5))
sns.histplot(df_sizes['Width'], color='skyblue', kde=True, label='Width', bins=30)
sns.histplot(df_sizes['Height'], color='orange', kde=True, label='Height', bins=30)
plt.title("Image Resolution Distribution")
plt.xlabel("Pixels")
plt.legend()
plt.tight_layout()
plt.show()

# Step 5: Print summary statistics
print("Class Distribution:\n", df_counts2)
print("\nImage Resolution Stats:\n", df_sizes.describe())

In [ ]:
plt.figure(figsize=(7, 7))
plt.pie(df_counts['Image_Count'], labels=df_counts2['Tumor_Type'], autopct='%1.1f%%', startangle=140, colors=sns.color_palette('Set2'))
plt.title("Tumor Class Distribution (Pie Chart)")
plt.tight_layout()
plt.show()

In [ ]:
data_dir = "/content/brain"
data_dir = pathlib.Path(data_dir)

In [ ]:
# Define transformations (resize, normalize, augment train)
import pathlib

data_dir='/content/drive/MyDrive/Tumour'
#train_dir = os.path.join(dataset_dir, 'train')
data_dir = pathlib.Path(data_dir)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(30),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Random shift (10%)
    transforms.ColorJitter(brightness=0.2, contrast=0.2),   # Random brightness & contrast
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),    # Random zoom (scale)
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

In [ ]:
valid_dir = os.path.join(data_dir, 'valid')
test_dir = os.path.join(data_dir, 'test')
train_dir = os.path.join(data_dir, 'train')
train_dataset = ImageFolder(train_dir, transform=transform)
valid_dataset = ImageFolder(valid_dir, transform=transform)
test_dataset = ImageFolder(test_dir, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Class names
classes = train_dataset.classes
print("Train Classes:", classes)
classes1 = valid_dataset.classes
print("valid Classes:", classes1)
classes2 = test_dataset.classes
print("test Classes:", classes2)

In [ ]:
#visualize augmented images from training dataset.
def imshow(img):
    img = img.numpy().transpose((1, 2, 0))
    img = img * 0.5 + 0.5  # unnormalize
    plt.imshow(img)
    plt.axis('off')
    plt.show()

# Visualize one batch
data_iter = iter(train_loader)
images, labels = next(data_iter)

for i in range(5):
    imshow(images[i])

In [ ]:
#visualize augmented images from valid dataset.
def imshow(img):
    img = img.numpy().transpose((1, 2, 0))
    img = img * 0.5 + 0.5  # unnormalize
    plt.imshow(img)
    plt.axis('off')
    plt.show()

# Visualize one batch
data_iter = iter(valid_loader)
images, labels = next(data_iter)

for i in range(5):
    imshow(images[i])

In [ ]:
#visualize augmented images from test dataset.
def imshow(img):
    img = img.numpy().transpose((1, 2, 0))
    img = img * 0.5 + 0.5  # unnormalize
    plt.imshow(img)
    plt.axis('off')
    plt.show()

# Visualize one batch
data_iter = iter(test_loader)
images, labels = next(data_iter)

for i in range(5):
    imshow(images[i])

In [ ]:
import torchvision

train_set = torchvision.datasets.ImageFolder(data_dir.joinpath('train'), transform=transform)
train_set.transform
val_set = torchvision.datasets.ImageFolder(data_dir.joinpath('valid'), transform=transform)
val_set.transform
test_set = torchvision.datasets.ImageFolder(data_dir.joinpath('test'), transform=transform)
test_set.transform

In [ ]:
train_set

In [ ]:
val_set

In [ ]:
test_set

In [ ]:
img, label = train_set[1000]
plt.imshow(img.permute(1,2,0))

In [ ]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size=batch_size, shuffle=True,num_workers=2, pin_memory=True)

In [ ]:
next(iter(train_loader))[0].shape

In [ ]:
next(iter(test_loader))[0].shape

In [ ]:
next(iter(val_loader))[0].shape

#### Model building
#### Custom CNN

In [ ]:
class CNNModel(nn.Module):
  def __init__(self):
    super(CNNModel, self).__init__()

    #Conv 1: 224x224 input
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=0) # (224-5+1)/1 = 220

    #Max Pool 1: 220x220 input
    self.maxpool1 = nn.MaxPool2d(kernel_size=2) # 220/2 = 110

    #Conv 2: 110x110 input
    self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0) # (110-5+1)/1 = 106

    #Max Pool 2: 106x106 input
    self.maxpool2 = nn.MaxPool2d(kernel_size=2) # 106/2 = 53

    #Conv 3: 53x53 input
    self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=0) # (53-5+1)/1 = 49

    #Max Pool 3: 49x49 input
    self.maxpool3 = nn.MaxPool2d(kernel_size=2) # 49/2 = 24 (floor)

    #Conv 4: 24x24 input
    self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=1, padding=0) # (24-5+1)/1 = 20

    #Max Pool 4: 20x20 input
    self.maxpool4 = nn.MaxPool2d(kernel_size=2) # 20/2 = 10

    #Activation Function
    self.leakyRelu = nn.LeakyReLU()

    # Calculate flattened size dynamically
    self._to_linear = 128 * 10 * 10 # Output channels of last conv * final spatial dimensions

    #Fully connected Layer 1
    self.fc1 = nn.Linear(self._to_linear, 1024)

    #Fully connected Layer 2
    self.fc2 = nn.Linear(1024, len(classes)) # Use len(classes) for the number of output classes

  def forward(self, x):
    #Layer 1
    out = self.leakyRelu(self.conv1(x))
    out = self.maxpool1(out)

    #Layer 2
    out = self.leakyRelu(self.conv2(out))
    out = self.maxpool2(out)

    #Layer 3
    out = self.leakyRelu(self.conv3(out))
    out = self.maxpool3(out)

    #Layer 4
    out = self.leakyRelu(self.conv4(out))
    out = self.maxpool4(out)

    #flatten
    out = out.view(out.size(0), -1)

    # Linear Function
    out = self.leakyRelu(self.fc1(out))
    out = self.fc2(out)

    return out

In [ ]:
# Training CNN
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
import torch.optim as optim

# Class names (moved from cell vQu2iVQK8iEl)
train_dataset = ImageFolder("/content/drive/MyDrive/Tumour/train", transform=transform) # Assuming transform is defined
classes = train_dataset.classes


num_epochs = 10
model = CNNModel()
error = nn.CrossEntropyLoss()

lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loss_list = []
val_loss_list = []
test_loss_list = []
accuracy_list = []

# Move model and criterion to device
model.to(device)
error.to(device)

for epoch in range(num_epochs):
    train_loss = 0
    val_loss = 0
    test_loss = 0
    correct_train = 0
    total_train = 0
    correct_val = 0
    total_val = 0
    correct_test = 0
    total_test = 0

    # Training loop
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = error(outputs, labels)
        train_loss += loss.item() * images.size(0) # Multiply by batch size for correct average later

        loss.backward()
        optimizer.step()
        # print("Loss in Iteration:", i, ":", loss.item()) # Optional: print loss per iteration

        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()


    # Validation loop
    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            v_loss = error(outputs, labels)
            val_loss += v_loss.item() * images.size(0) # Multiply by batch size

            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    # Test loop
    model.eval()
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs1 = model(images) # Use outputs1 for test
            tes_loss = error(outputs1, labels) # Use outputs1 here
            test_loss += tes_loss.item() * images.size(0) # Multiply by batch size

            _, predicted1 = torch.max(outputs1.data, 1) # Use predicted1 for test
            total_test += labels.size(0)
            correct_test += (predicted1 == labels).sum().item()


    # Calculate average losses and accuracies
    avg_train_loss = train_loss / total_train
    avg_val_loss = val_loss / total_val
    avg_test_loss = test_loss / total_test

    train_accuracy = 100 * correct_train / total_train
    val_accuracy = 100 * correct_val / total_val
    test_accuracy = 100 * correct_test / total_test


    # Append to lists
    train_loss_list.append(avg_train_loss)
    val_loss_list.append(avg_val_loss)
    test_loss_list.append(avg_test_loss)
    accuracy_list.append(train_accuracy) # Store train, val, test accuracy for the epoch
    accuracy_list.append(val_accuracy)
    accuracy_list.append(test_accuracy)


    # Step the scheduler based on validation loss
    scheduler.step(avg_val_loss)

    print(f"Epoch: {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Valid Loss: {avg_val_loss:.4f}, Test Loss: {avg_test_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Valid Acc: {val_accuracy:.2f}%, Test Acc: {test_accuracy:.2f}%")

#In the case of CNN i got the test accc and valid acc is 73%. which is a littile bit low and losses are there. So pretrained the model

In [ ]:
# Assuming 'classes' is defined, e.g., as a list of class names
# For demonstration purposes, let's define a dummy 'classes'
classes = ['class','class1', 'class2', 'class3']

In [ ]:
num_classes=len(classes)

###Transfer Learning

###Load pretrained models (Example : ResNet50)


In [ ]:
resnet_model = models.resnet50(pretrained=True)

# Freeze all layers
for param in resnet_model.parameters():
    param.requires_grad = False

# Replace the final layer
resnet_model.fc = nn.Sequential(
    nn.Linear(resnet_model.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512, num_classes)
)

resnet_model = resnet_model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet_model.fc.parameters(), lr=0.001)

In [ ]:
#Train the Model
def train_model(model, train_loader, val_loader,test_loader, epochs=10):
    model.train()

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        acc = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss:.4f}, Accuracy: {acc:.2f}%")

In [ ]:
train_model(resnet_model, train_loader, valid_loader,test_loader, epochs=10)

#In the case of resnet 50 i got the  Accuracy  84% and loss is really very less here. Pretrained the model to get very high accuracy

######Model Training


#####Train both custom CNN and transfer learning models.


In [ ]:
#Define Custom CNN Architecture

import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()

        self.conv_block1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.conv_block3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(128 * 28 * 28, 256)  # assuming input image size is 224x224
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

In [ ]:
#Unified Training Function (for Custom CNN or ResNet)

import torch
import numpy as np
import copy
import os

class EarlyStopping:
    def __init__(self, patience=3, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_loss = np.inf # Changed np.Inf to np.inf
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            print("EarlyStopping counter: {self.counter} / {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True

def save_checkpoint(model, path='best_model.pth'):
    torch.save(model.state_dict(), path)
    print(f"Model saved at {path}")

def train_with_callbacks(model, criterion, optimizer, train_loader, val_loader, device,
                         num_epochs=25, early_stopping=None, checkpoint_path='best_model.pth'):

    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_loss = float('inf')
    best_test_loss = float('inf')  # Initialize best_test_loss

    train_loss_history = []
    val_loss_history = []
    test_loss_history =[]
    train_acc_history = []
    val_acc_history = []
    test_acc_history = []

    for epoch in range(num_epochs):
        print(f"\n Epoch {epoch+1}/{num_epochs}")

        model.train()
        train_loss, correct, total = 0.0, 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        epoch_train_loss = train_loss / total
        epoch_train_acc = correct / total
        train_loss_history.append(epoch_train_loss)
        train_acc_history.append(epoch_train_acc)

        # Validation phase
        model.eval()
        val_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        epoch_val_loss = val_loss / total
        epoch_val_acc = correct / total
        val_loss_history.append(epoch_val_loss)
        val_acc_history.append(epoch_val_acc)



        # Test phase
        model.eval()
        test_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in test_loader: # Changed from val_loader to test_loader
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                test_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        epoch_test_loss = test_loss / total
        epoch_test_acc = correct / total
        test_loss_history.append(epoch_test_loss) # Corrected append
        test_acc_history.append(epoch_test_acc) # Corrected append

        print(f"Train Loss: {epoch_train_loss:.4f}, Acc: {epoch_train_acc:.4f}")
        print(f"Val   Loss: {epoch_val_loss:.4f}, Acc: {epoch_val_acc:.4f}")
        print(f"Test   Loss: {epoch_test_loss:.4f}, Acc: {epoch_test_acc:.4f}")


        # Checkpoint
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            save_checkpoint(model, checkpoint_path)
        if epoch_test_loss < best_test_loss:
            best_test_loss = epoch_test_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            save_checkpoint(model, checkpoint_path)


        # Early Stopping
        if early_stopping:
            early_stopping(epoch_val_loss)
            if early_stopping.early_stop:
                print("Early stopping triggered based on validation loss.")
                break
        if early_stopping:
            early_stopping(epoch_test_loss)
            if early_stopping.early_stop:
                print("Early stopping triggered based on test loss.")
                break


    model.load_state_dict(best_model_wts)
    return model, train_loss_history, val_loss_history,test_loss_history, train_acc_history, val_acc_history,test_acc_history

In [ ]:
#Train Both Models
# Custom CNN (Example)
#num_classes = len(classes)  # classes = train_dataset.classes
#cnn_model = CNNModel(num_classes).to(device)
cnn_model = CustomCNN(num_classes).to(device)

optimizer_cnn = torch.optim.Adam(cnn_model.parameters(), lr=1e-4)
early_stopper = EarlyStopping(patience=5)

cnn_model, cnn_train_loss, cnn_val_loss,cnn_test_loss, cnn_train_acc, cnn_val_acc,cnn_test_acc = train_with_callbacks(
    cnn_model,
    nn.CrossEntropyLoss(),
    optimizer_cnn,
    train_loader,
    valid_loader,
    device,
    num_epochs=10,
    early_stopping=early_stopper,
    checkpoint_path='cnn_best_model.pth'
)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset

# Assuming you have a custom dataset and loaders, these are placeholders
# for demonstration. Replace them with your actual dataset and loaders.
class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Placeholder for your data and loaders
# You should define your actual train_dataset, valid_dataset, etc.
# For example:
# from torchvision.datasets import ImageFolder
# from torch.utils.data import DataLoader
# train_dataset = ImageFolder('path/to/train', transform=your_transform)
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# num_classes = len(train_dataset.classes)
num_classes = 10 # Example number of classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---
# 1. Load pretrained ResNet50
# ---
# Use a more modern weight as it often performs better
weights = models.ResNet50_Weights.IMAGENET1K_V2
resnet = models.resnet50(weights=weights)

# ---
# 2. Adjusting the fine-tuning strategy
# ---
# Freeze all layers first
for param in resnet.parameters():
    param.requires_grad = False

# Unfreeze the last two blocks (layer3 and layer4) for more effective fine-tuning.
# This provides a better balance between feature reuse and task-specific learning.
# Fine-tuning only layer4 and fc might not be enough to learn the new features well.
for name, param in resnet.named_parameters():
    if "layer3" in name or "layer4" in name or "fc" in name:
        param.requires_grad = True

# ---
# 3. Replace final layer for custom classification
# ---
# This part is correct, but let's make it a more robust replacement using Sequential
resnet.fc = nn.Sequential(
    nn.Linear(resnet.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, num_classes)
)

# ---
# 4. Move model to GPU
# ---
resnet = resnet.to(device)

# ---
# 5. Use optimizer with a better approach
# ---
# Use a more sophisticated optimizer like AdamW which often performs better than Adam
# and a lower learning rate for the frozen layers while a higher one for the new layers.
# We also use a separate optimizer for the new layers to have more control.
optimizer_resnet = optim.AdamW(
    filter(lambda p: p.requires_grad, resnet.parameters()),
    lr=1e-4,
    weight_decay=1e-5
)

# ---
# 6. Define loss function
# ---
criterion = nn.CrossEntropyLoss()

# ---
# 7. Add learning rate scheduler
# ---
# Reduce the learning rate as training progresses to avoid overshooting the minimum.
# This helps the model converge more smoothly and improves accuracy.
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer_resnet, 'max', patience=5, factor=0.1)

# ---
# 8. Define a robust training function with data augmentation
# ---
# Data augmentation is critical for preventing overfitting, especially with smaller datasets.
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Your custom train_model function should be updated to include the scheduler logic.
# The `train_model` function you're using is not provided, but here is an example
# of how the training loop should be structured to use the scheduler.
def improved_train_model(model, train_loader, val_loader, optimizer, criterion, scheduler, epochs, device):
    best_val_accuracy = 0.0
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation phase
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, Validation Accuracy: {val_accuracy:.2f}%")

        # Step the scheduler based on validation accuracy
        scheduler.step(val_accuracy)

        # Save the best model
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), 'resnet_best_model.pth')

    return model

In [ ]:
improved_train_model(
  model=resnet,
 train_loader=train_loader,
 val_loader=valid_loader,
  optimizer=optimizer_resnet,
  criterion=criterion,
  scheduler=scheduler,
  epochs=10,
  device=device
)

#Pretrained model resnet50 i got 94% accuracy which is good. Here i got more accuracy uesing resnet50 compare to custom CNN

####Model Evaluation


###Evaluate models using metrics like accuracy, precision, recall, F1-score, and confusion matrix.


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

def evaluate_model(model, data_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Metrics
    acc = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

    print("Accuracy:", round(acc, 4))
    print("Precision:", round(precision, 4))
    print("Recall:", round(recall, 4))
    print("F1 Score:", round(f1, 4))
    print("\nConfusion Matrix:\n", confusion_matrix(all_labels, all_preds))
    print("\nClassification Report:\n", classification_report(all_labels, all_preds))

In [ ]:
evaluate_model(model, test_loader,device)

In [ ]:
evaluate_model(resnet, test_loader,device)

#Model Comparison


# Compare results of custom CNN vs pretrained models.


#Identify the most accurate, efficient, and reliable model for deployment.


In [ ]:
import matplotlib.pyplot as plt

models = ['Custom CNN', 'ResNet50']
accuracies = [73, 96]
f1_scores = [70, 95]

plt.figure(figsize=(8,5))
bar_width = 0.35
x = range(len(models))

plt.bar(x, accuracies, width=bar_width, label='Accuracy')
plt.bar([i + bar_width for i in x], f1_scores, width=bar_width, label='F1-score')
plt.xticks([i + bar_width / 2 for i in x], models)
plt.ylabel('Score (%)')
plt.title('Model Comparison')
plt.legend()
plt.show()

The graph titled "Model Comparison" showcases the performance of two deep learning models—Custom CNN and ResNet50—based on accuracy and F1-score metrics. Custom CNN reaches approximately 73% accuracy and70% F1-score, while ResNet50 significantly outperforms it with about 96% accuracy and 95% F1-score. This visualization clearly highlights ResNet50’s superior performance, making it the stronger choice for this classification task.

#### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count

In [ ]:
# Visualizing the missing values

### What did you know about your dataset?

Answer Here

## ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns

In [ ]:
# Dataset Describe

### Variables Description

Answer Here

### Check Unique Values for each variable.

In [ ]:
# Check Unique Values for each variable.

## 3. ***Data Wrangling***

### Data Wrangling Code

In [ ]:
# Write your code to make your dataset analysis ready.

### What all manipulations have you done and insights you found?

Answer Here.

## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

#### Chart - 1

In [ ]:
# Chart - 1 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 2

In [ ]:
# Chart - 2 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 3

In [ ]:
# Chart - 3 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 4

In [ ]:
# Chart - 4 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 5

In [ ]:
# Chart - 5 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 6

In [ ]:
# Chart - 6 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 7

In [ ]:
# Chart - 7 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 8

In [ ]:
# Chart - 8 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 9

In [ ]:
# Chart - 9 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 10

In [ ]:
# Chart - 10 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 11

In [ ]:
# Chart - 11 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 12

In [ ]:
# Chart - 12 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 13

In [ ]:
# Chart - 13 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 14 - Correlation Heatmap

In [ ]:
# Correlation Heatmap visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

#### Chart - 15 - Pair Plot

In [ ]:
# Pair Plot visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

## ***5. Hypothesis Testing***

### Based on your chart experiments, define three hypothetical statements from the dataset. In the next three questions, perform hypothesis testing to obtain final conclusion about the statements through your code and statistical testing.

Answer Here.

### Hypothetical Statement - 1

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Answer Here.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value

##### Which statistical test have you done to obtain P-Value?

Answer Here.

##### Why did you choose the specific statistical test?

Answer Here.

### Hypothetical Statement - 2

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Answer Here.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value

##### Which statistical test have you done to obtain P-Value?

Answer Here.

##### Why did you choose the specific statistical test?

Answer Here.

### Hypothetical Statement - 3

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Answer Here.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value

##### Which statistical test have you done to obtain P-Value?

Answer Here.

##### Why did you choose the specific statistical test?

Answer Here.

## ***6. Feature Engineering & Data Pre-processing***

### 1. Handling Missing Values

In [ ]:
# Handling Missing Values & Missing Value Imputation

#### What all missing value imputation techniques have you used and why did you use those techniques?

Answer Here.

### 2. Handling Outliers

In [ ]:
# Handling Outliers & Outlier treatments

##### What all outlier treatment techniques have you used and why did you use those techniques?

Answer Here.

### 3. Categorical Encoding

In [ ]:
# Encode your categorical columns

#### What all categorical encoding techniques have you used & why did you use those techniques?

Answer Here.

### 4. Textual Data Preprocessing
(It's mandatory for textual dataset i.e., NLP, Sentiment Analysis, Text Clustering etc.)

#### 1. Expand Contraction

In [ ]:
# Expand Contraction

#### 2. Lower Casing

In [ ]:
# Lower Casing

#### 3. Removing Punctuations

In [ ]:
# Remove Punctuations

#### 4. Removing URLs & Removing words and digits contain digits.

In [ ]:
# Remove URLs & Remove words and digits contain digits

#### 5. Removing Stopwords & Removing White spaces

In [ ]:
# Remove Stopwords

In [ ]:
# Remove White spaces

#### 6. Rephrase Text

In [ ]:
# Rephrase Text

#### 7. Tokenization

In [ ]:
# Tokenization

#### 8. Text Normalization

In [ ]:
# Normalizing Text (i.e., Stemming, Lemmatization etc.)

##### Which text normalization technique have you used and why?

Answer Here.

#### 9. Part of speech tagging

In [ ]:
# POS Taging

#### 10. Text Vectorization

In [ ]:
# Vectorizing Text

##### Which text vectorization technique have you used and why?

Answer Here.

### 4. Feature Manipulation & Selection

#### 1. Feature Manipulation

In [ ]:
# Manipulate Features to minimize feature correlation and create new features

#### 2. Feature Selection

In [ ]:
# Select your features wisely to avoid overfitting

##### What all feature selection methods have you used  and why?

Answer Here.

##### Which all features you found important and why?

Answer Here.

### 5. Data Transformation

#### Do you think that your data needs to be transformed? If yes, which transformation have you used. Explain Why?

In [ ]:
# Transform Your data

### 6. Data Scaling

In [ ]:
# Scaling your data

##### Which method have you used to scale you data and why?

### 7. Dimesionality Reduction

##### Do you think that dimensionality reduction is needed? Explain Why?

Answer Here.

In [ ]:
# DImensionality Reduction (If needed)

##### Which dimensionality reduction technique have you used and why? (If dimensionality reduction done on dataset.)

Answer Here.

### 8. Data Splitting

In [ ]:
# Split your data to train and test. Choose Splitting ratio wisely.

##### What data splitting ratio have you used and why?

Answer Here.

### 9. Handling Imbalanced Dataset

##### Do you think the dataset is imbalanced? Explain Why.

Answer Here.

In [ ]:
# Handling Imbalanced Dataset (If needed)

##### What technique did you use to handle the imbalance dataset and why? (If needed to be balanced)

Answer Here.

## ***7. ML Model Implementation***

### ML Model - 1

In [ ]:
# ML Model - 1 Implementation

# Fit the Algorithm

# Predict on the model

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 1 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

### ML Model - 2

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 1 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

#### 3. Explain each evaluation metric's indication towards business and the business impact pf the ML model used.

Answer Here.

### ML Model - 3

In [ ]:
# ML Model - 3 Implementation

# Fit the Algorithm

# Predict on the model

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 3 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

### 1. Which Evaluation metrics did you consider for a positive business impact and why?

Answer Here.

### 2. Which ML model did you choose from the above created models as your final prediction model and why?

Answer Here.

### 3. Explain the model which you have used and the feature importance using any model explainability tool?

Answer Here.

## ***8.*** ***Future Work (Optional)***

### 1. Save the best performing ml model in a pickle file or joblib file format for deployment process.


In [ ]:
# Save the File

### 2. Again Load the saved model file and try to predict unseen data for a sanity check.


In [ ]:
# Load the File and predict unseen data.

### ***Congrats! Your model is successfully created and ready for deployment on a live server for a real user interaction !!!***

# **Conclusion**

Write the conclusion here.

### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***

In [ ]:
!pip install splitfolders

In [ ]:
import os

# List the contents of the output directory
split_dir = "brain_split"
if os.path.exists(split_dir):
    print(f"Contents of '{split_dir}':")
    for item in os.listdir(split_dir):
        print(f"- {item}")
else:
    print(f"Directory '{split_dir}' not found.")